# Variying parameters

This notebook contains code to generate some of the data in the paper "Optimizing testing policies for detecting COVID-19 within institutions"
by Boaz Barak, Mor Nitzan, Neta Ravid Tannenbaum, and Janni Yuval.

## Load dependencies

In [ ]:
import networkx
import numpy as np
import pandas as pd
import os

In [2]:
from models import *
from helpers import *

### Parameter setup



In [ ]:
G_normal , G_quarantine = make_graphs(numNodes = 1000, m=9, scale = 100 , plot_degree = True)


In [4]:
base = {
    "G": {"numNodes" : 1000, "m" : 9, "scale":100 , "plot_degree": False}, #Network adjacency matrix (numpy array) or Networkx graph object.
    "beta":0.2,# Rate of transmission (exposure) (global)
    "sigma":1/5.2, # Rate of infection (upon exposure)
    "gamma" : 1/14, # Rate of recovery (upon infection)
    "mu_I": 0.0004, # Rate of infection-related death
    "mu_0": 0,# Rate of baseline death
    "nu":0, # Rate of baseline birth
     "xi":0, # Rate of re-susceptibility (upon recovery)
     "p":0.5, # Probability of interaction outside adjacent nodes
     "Q": None, # Quarantine adjacency matrix (numpy array) or Networkx graph object.
     "beta_D": 0, # Rate of transmission (exposure) for individuals with detected infections (global), changed from 0.155 to set so detected individuals can't transmit
     "sigma_D": 1/5.2,   # Rate of infection (upon exposure) for individuals with detected infections
     "gamma_D":1/14, # Rate of recovery (upon infection) for individuals with detected infections
      "mu_D":0.0004, # Rate of infection-related death for individuals with detected infections
      "theta_E":0, # probability that exposed person is tested (i.e., becomes symptomatic)
      "theta_I":0, # probability that infected person is tested (i.e., becomes symptomatic)
      "phi_E":0, # each detected contact adds phi_E chance for probability of infection for exposed individual
      "phi_I": 0, # each detected contact adds phi_I chance for probability of infection for infected individual
      "psi_E": 1.0, #Probability of positive test results for exposed individuals. 1 =  no false negatives
      "psi_I": 1.0, # Probability of positive test results for exposed individuals. 1 = no false negatives
       "q":0.0, # Probability of quarantined individuals interaction outside adjacent nodes.   changed to 0 to make it strong quarantine
       "initI": 0, # Init number of infectious individuals
       "initE":0, # Init number of exposed  individuals
       "initD_E":0, # Init number of detected exposed individuals
       "initD_I":0, # Init number of detected infectious individuals
       "initR":0, # Init number of recovered individuals
      "initF":0, # Init number of infection -related fatalities
      "p_extern": 1/1000, # Rate of spontanous (external) infection
      "p_periodic": 0, # Fraction of people tested in a periodic subsample tests
      "period": 1, # Period for periodic testing
      "batch" : True, # Test people in batch (True) or test each person with probability p_periodic (False)
      "min_time" : 0, # Minimum time to pass between two tests for the same individual
      "store_Xseries" : False, # Store the full data per node
      "count_non_random" : False, #  True: count tests done to symptomatic or traced people
      "T": 84, # time to run the execution
      "verbose": False, # print log messages during run
       "checkpoints": None, # checkpoints (see SIERPlus documentation - we don't use this)
       "stopping": "1st", # function for stopping policy '1st' means stop at first detection
       "policy": None, # policy function for adaptive policies
       "policyInterval": 1, # period to apply policy function
        "type": "[UNKNOWN]", # type of simulation - useful for aggragating data
        "runTillEnd": True, # True - don't stop even if zero infections (makes sense when external infections > 0),
        "test_recovered": True, # Test people in "recovered" state - makes sense when we stop at first detection, not so much otherwise
        "initT": 0 # Initial time step
}

orig_base = dict(base)

In [5]:
# Testing symptomatic and tracing (unused when we stop at first detection)

symp_ratio = 0.2 # fraction of people that develop symptoms significant enough to be tested and isolated sometime during the time they spend in infected or exposed state
# this number is necessarily guessed, since it depends heavily on the demographics of the workplace
# as well as policies in place for self isolation and testing.
#symp_affin  =symp_ratio* base["sigma"] * base["gamma"] / ( base["gamma"] + base["sigma"] )

symp_affin = 0.2 * base["gamma"]

# Chosen so that symp_affin/sigma + symp_affin/sigma = symp_ratio

# probability of developing infection

symptomatic = {
    "theta_E" : 0 , # probability that exposed person is tested (i.e., becomes symptomatic)
    "theta_I" : symp_affin  # probability that infected person is tested (i.e., becomes symptomatic)
}


# a/sigma = 0.2*(1/sigma / (1/gamma + 1/delta)) = gamma*sigma/

# Contact tracing - a detected E/I neighbor increases the chance to be tested by 50%
tracing =  {
    "phi_E": 0.8,
    "phi_I": 0.8
}

# Uncomment lines below to include tracing & testing of symptomatic people

#base.update(tracing)
#base.update(symptomatic)

We set a false negative rate of 67% for exposed individuals and 20% for infected individuals.

In [ ]:
# False negatives

base.update({
    "psi_E": 0.33, #Probability of positive test results for exposed individuals. 1 =  no false negatives
    "psi_I": 0.8, # Probability of positive test results for exposed individuals. 1 = no false negatives
})

#  Experiments



In [7]:
def test(days,batches):
    """Modify parameters to test 1/batches of the population every days/batches days"""
    return { "p_periodic" : 1/batches, "period": days/batches }

def product(A,B):
    return [(a,b) for a in A for b in B]

In [8]:

to_do =product([28],[1,2,3,28]) + \
       product([14],[1,2,3,14])


    

base["T"] = 108

In [9]:
to_do = [(28,1),(28,4)]

settings = [{**base, **test(d,k), "type":f"{d}/{k}" } for (d,k) in to_do ]



settings += [{**base , "type": "No testing (56 days)", "T": 56 } ]



In [10]:
base

{'G': {'numNodes': 1000, 'm': 9, 'scale': 100, 'plot_degree': False},
 'beta': 0.2,
 'sigma': 0.1923076923076923,
 'gamma': 0.07142857142857142,
 'mu_I': 0.0004,
 'mu_0': 0,
 'nu': 0,
 'xi': 0,
 'p': 0.5,
 'Q': None,
 'beta_D': 0,
 'sigma_D': 0.1923076923076923,
 'gamma_D': 0.07142857142857142,
 'mu_D': 0.0004,
 'theta_E': 0,
 'theta_I': 0,
 'phi_E': 0,
 'phi_I': 0,
 'psi_E': 0.33,
 'psi_I': 0.8,
 'q': 0.0,
 'initI': 0,
 'initE': 0,
 'initD_E': 0,
 'initD_I': 0,
 'initR': 0,
 'initF': 0,
 'p_extern': 0.001,
 'p_periodic': 0,
 'period': 1,
 'batch': True,
 'min_time': 0,
 'store_Xseries': False,
 'count_non_random': False,
 'T': 108,
 'verbose': False,
 'checkpoints': None,
 'stopping': '1st',
 'policy': None,
 'policyInterval': 1,
 'type': '[UNKNOWN]',
 'runTillEnd': True,
 'test_recovered': True,
 'initT': 0}

## Get variants for p_extern, beta

In [11]:
from fractions import Fraction

std_graph =   {"numNodes" : 1000, "m" : 9, "scale":100 , "plot_degree": False}

options = [(beta,p,std_graph) for beta in np.linspace(0.025,0.3,15) for p in np.linspace(0.00005,0.001,15)]



In [12]:
def name(G):
    if G["m"] <=9:
        return "" 
    else: 
        return " (more edges)"

to_run = [] 
variants = {}
for beta,p,G in options:
    #print(beta,p,G)
    q = Fraction(p).limit_denominator(100000) if (p and (not isinstance(p,str))) else 0
    for d in settings:
        e = dict(d)
        
        v = f"beta={beta},pextern={q}"+name(G)
        if p==0:
            e["initI"] = 1
            v += " (1 initial infection)"
        if isinstance(p,str) and p[0]=='r':
            a = float(p[1:])
            e["initT"]=(0,a)
            e["initI"]=1
            e["p_extern"] = 0
            v += f" (random start 0-{a})"
        else:
            e["p_extern"] = p
        if e["type"][:15] != "Business closed":
            e["G"] = G
            e["beta"] = beta
            
        #e["verbose"] = True
        variants[v] = { "beta":beta, "p_extern": p, "G":G }
        e["variant"] =  v
        to_run += [e]
        
print(len(variants.keys()))
#variants

225


## This is the time consuming part


In [13]:
# commented out - this is just when debugging
#to_run = [{**base, "p_periodic":1/20, "period":1, "T":200, "p_extern":1/30000 , "type":"Trial", "verbose":False, "min_time":0}]

In [14]:
realizations = 400
#lview = None 
len(to_run)*realizations

270000

In [17]:
if not os.path.exists("data/"):
    os.makedirs("data")

timestamp = datetime.now().strftime('%Y%m%d_%H_%M_%S')
datadir = f"data/{timestamp}_{realizations}"
os.makedirs(datadir)
print(datadir)

data/20200623_12_17_42_400


In [18]:
def pline(L,n=None):
    T = []
    for a in L:
        if isinstance(a,float):
            if a > 0.001:
                T += [f"{a:.3f}"]
            else:
                q = Fraction(p).limit_denominator(100000) 
                T += [f"{q}"]
        else:
            T += [str(a) ]
    if not n:
        n = max([len(a)+2 for a in T])
    
    return "|".join([a.ljust(n) for a in T])+"\n"
                
        
    
readme = fr"""
# Description

The above is results of simulations done at {timestamp}

We use {len(settings)} parameter settings, each with {len(list(variants.keys()))} variants. We describe the main settings below:

We use {realizations} realizations, for a total of {realizations * len(to_run)} executions.

## Variants:

"""
for v in variants.keys():
    readme += f"   *  {v}\n"

readme += "## Settings \n"

keys = ["type", "p_periodic", "period"]
readme += pline(keys,25)
readme += "|".join(["-"*25]*len(keys))+"\n"
for s in settings:
    readme += pline([s["type"], s["p_periodic"],s["period"] ],25)

readme += f"""

## Baseline parameters
"""    

readme += pline(["parameter","value"],20)
readme += "|".join(["-"*20]*2)+"\n"
for k in base:
    readme += pline([k,base[k]],20)

with open(f"{datadir}/readme.md", 'a') as out:
    out.write(readme + '\n')

from IPython.display import display, Markdown

display(Markdown(readme))


# Description

The above is results of simulations done at 20200623_12_17_42

We use 3 parameter settings, each with 225 variants. We describe the main settings below:

We use 400 realizations, for a total of 270000 executions.

## Variants:

   *  beta=0.025,pextern=1/20000
   *  beta=0.025,pextern=7/59394
   *  beta=0.025,pextern=13/70000
   *  beta=0.025,pextern=3/11831
   *  beta=0.025,pextern=9/28000
   *  beta=0.025,pextern=26/66789
   *  beta=0.025,pextern=2/4375
   *  beta=0.025,pextern=21/40000
   *  beta=0.025,pextern=59/99518
   *  beta=0.025,pextern=37/56000
   *  beta=0.025,pextern=51/70000
   *  beta=0.025,pextern=38/47713
   *  beta=0.025,pextern=81/93719
   *  beta=0.025,pextern=64/68659
   *  beta=0.025,pextern=1/1000
   *  beta=0.04464285714285714,pextern=1/20000
   *  beta=0.04464285714285714,pextern=7/59394
   *  beta=0.04464285714285714,pextern=13/70000
   *  beta=0.04464285714285714,pextern=3/11831
   *  beta=0.04464285714285714,pextern=9/28000
   *  beta=0.04464285714285714,pextern=26/66789
   *  beta=0.04464285714285714,pextern=2/4375
   *  beta=0.04464285714285714,pextern=21/40000
   *  beta=0.04464285714285714,pextern=59/99518
   *  beta=0.04464285714285714,pextern=37/56000
   *  beta=0.04464285714285714,pextern=51/70000
   *  beta=0.04464285714285714,pextern=38/47713
   *  beta=0.04464285714285714,pextern=81/93719
   *  beta=0.04464285714285714,pextern=64/68659
   *  beta=0.04464285714285714,pextern=1/1000
   *  beta=0.06428571428571428,pextern=1/20000
   *  beta=0.06428571428571428,pextern=7/59394
   *  beta=0.06428571428571428,pextern=13/70000
   *  beta=0.06428571428571428,pextern=3/11831
   *  beta=0.06428571428571428,pextern=9/28000
   *  beta=0.06428571428571428,pextern=26/66789
   *  beta=0.06428571428571428,pextern=2/4375
   *  beta=0.06428571428571428,pextern=21/40000
   *  beta=0.06428571428571428,pextern=59/99518
   *  beta=0.06428571428571428,pextern=37/56000
   *  beta=0.06428571428571428,pextern=51/70000
   *  beta=0.06428571428571428,pextern=38/47713
   *  beta=0.06428571428571428,pextern=81/93719
   *  beta=0.06428571428571428,pextern=64/68659
   *  beta=0.06428571428571428,pextern=1/1000
   *  beta=0.08392857142857141,pextern=1/20000
   *  beta=0.08392857142857141,pextern=7/59394
   *  beta=0.08392857142857141,pextern=13/70000
   *  beta=0.08392857142857141,pextern=3/11831
   *  beta=0.08392857142857141,pextern=9/28000
   *  beta=0.08392857142857141,pextern=26/66789
   *  beta=0.08392857142857141,pextern=2/4375
   *  beta=0.08392857142857141,pextern=21/40000
   *  beta=0.08392857142857141,pextern=59/99518
   *  beta=0.08392857142857141,pextern=37/56000
   *  beta=0.08392857142857141,pextern=51/70000
   *  beta=0.08392857142857141,pextern=38/47713
   *  beta=0.08392857142857141,pextern=81/93719
   *  beta=0.08392857142857141,pextern=64/68659
   *  beta=0.08392857142857141,pextern=1/1000
   *  beta=0.10357142857142856,pextern=1/20000
   *  beta=0.10357142857142856,pextern=7/59394
   *  beta=0.10357142857142856,pextern=13/70000
   *  beta=0.10357142857142856,pextern=3/11831
   *  beta=0.10357142857142856,pextern=9/28000
   *  beta=0.10357142857142856,pextern=26/66789
   *  beta=0.10357142857142856,pextern=2/4375
   *  beta=0.10357142857142856,pextern=21/40000
   *  beta=0.10357142857142856,pextern=59/99518
   *  beta=0.10357142857142856,pextern=37/56000
   *  beta=0.10357142857142856,pextern=51/70000
   *  beta=0.10357142857142856,pextern=38/47713
   *  beta=0.10357142857142856,pextern=81/93719
   *  beta=0.10357142857142856,pextern=64/68659
   *  beta=0.10357142857142856,pextern=1/1000
   *  beta=0.12321428571428569,pextern=1/20000
   *  beta=0.12321428571428569,pextern=7/59394
   *  beta=0.12321428571428569,pextern=13/70000
   *  beta=0.12321428571428569,pextern=3/11831
   *  beta=0.12321428571428569,pextern=9/28000
   *  beta=0.12321428571428569,pextern=26/66789
   *  beta=0.12321428571428569,pextern=2/4375
   *  beta=0.12321428571428569,pextern=21/40000
   *  beta=0.12321428571428569,pextern=59/99518
   *  beta=0.12321428571428569,pextern=37/56000
   *  beta=0.12321428571428569,pextern=51/70000
   *  beta=0.12321428571428569,pextern=38/47713
   *  beta=0.12321428571428569,pextern=81/93719
   *  beta=0.12321428571428569,pextern=64/68659
   *  beta=0.12321428571428569,pextern=1/1000
   *  beta=0.14285714285714282,pextern=1/20000
   *  beta=0.14285714285714282,pextern=7/59394
   *  beta=0.14285714285714282,pextern=13/70000
   *  beta=0.14285714285714282,pextern=3/11831
   *  beta=0.14285714285714282,pextern=9/28000
   *  beta=0.14285714285714282,pextern=26/66789
   *  beta=0.14285714285714282,pextern=2/4375
   *  beta=0.14285714285714282,pextern=21/40000
   *  beta=0.14285714285714282,pextern=59/99518
   *  beta=0.14285714285714282,pextern=37/56000
   *  beta=0.14285714285714282,pextern=51/70000
   *  beta=0.14285714285714282,pextern=38/47713
   *  beta=0.14285714285714282,pextern=81/93719
   *  beta=0.14285714285714282,pextern=64/68659
   *  beta=0.14285714285714282,pextern=1/1000
   *  beta=0.16249999999999998,pextern=1/20000
   *  beta=0.16249999999999998,pextern=7/59394
   *  beta=0.16249999999999998,pextern=13/70000
   *  beta=0.16249999999999998,pextern=3/11831
   *  beta=0.16249999999999998,pextern=9/28000
   *  beta=0.16249999999999998,pextern=26/66789
   *  beta=0.16249999999999998,pextern=2/4375
   *  beta=0.16249999999999998,pextern=21/40000
   *  beta=0.16249999999999998,pextern=59/99518
   *  beta=0.16249999999999998,pextern=37/56000
   *  beta=0.16249999999999998,pextern=51/70000
   *  beta=0.16249999999999998,pextern=38/47713
   *  beta=0.16249999999999998,pextern=81/93719
   *  beta=0.16249999999999998,pextern=64/68659
   *  beta=0.16249999999999998,pextern=1/1000
   *  beta=0.1821428571428571,pextern=1/20000
   *  beta=0.1821428571428571,pextern=7/59394
   *  beta=0.1821428571428571,pextern=13/70000
   *  beta=0.1821428571428571,pextern=3/11831
   *  beta=0.1821428571428571,pextern=9/28000
   *  beta=0.1821428571428571,pextern=26/66789
   *  beta=0.1821428571428571,pextern=2/4375
   *  beta=0.1821428571428571,pextern=21/40000
   *  beta=0.1821428571428571,pextern=59/99518
   *  beta=0.1821428571428571,pextern=37/56000
   *  beta=0.1821428571428571,pextern=51/70000
   *  beta=0.1821428571428571,pextern=38/47713
   *  beta=0.1821428571428571,pextern=81/93719
   *  beta=0.1821428571428571,pextern=64/68659
   *  beta=0.1821428571428571,pextern=1/1000
   *  beta=0.20178571428571423,pextern=1/20000
   *  beta=0.20178571428571423,pextern=7/59394
   *  beta=0.20178571428571423,pextern=13/70000
   *  beta=0.20178571428571423,pextern=3/11831
   *  beta=0.20178571428571423,pextern=9/28000
   *  beta=0.20178571428571423,pextern=26/66789
   *  beta=0.20178571428571423,pextern=2/4375
   *  beta=0.20178571428571423,pextern=21/40000
   *  beta=0.20178571428571423,pextern=59/99518
   *  beta=0.20178571428571423,pextern=37/56000
   *  beta=0.20178571428571423,pextern=51/70000
   *  beta=0.20178571428571423,pextern=38/47713
   *  beta=0.20178571428571423,pextern=81/93719
   *  beta=0.20178571428571423,pextern=64/68659
   *  beta=0.20178571428571423,pextern=1/1000
   *  beta=0.2214285714285714,pextern=1/20000
   *  beta=0.2214285714285714,pextern=7/59394
   *  beta=0.2214285714285714,pextern=13/70000
   *  beta=0.2214285714285714,pextern=3/11831
   *  beta=0.2214285714285714,pextern=9/28000
   *  beta=0.2214285714285714,pextern=26/66789
   *  beta=0.2214285714285714,pextern=2/4375
   *  beta=0.2214285714285714,pextern=21/40000
   *  beta=0.2214285714285714,pextern=59/99518
   *  beta=0.2214285714285714,pextern=37/56000
   *  beta=0.2214285714285714,pextern=51/70000
   *  beta=0.2214285714285714,pextern=38/47713
   *  beta=0.2214285714285714,pextern=81/93719
   *  beta=0.2214285714285714,pextern=64/68659
   *  beta=0.2214285714285714,pextern=1/1000
   *  beta=0.24107142857142852,pextern=1/20000
   *  beta=0.24107142857142852,pextern=7/59394
   *  beta=0.24107142857142852,pextern=13/70000
   *  beta=0.24107142857142852,pextern=3/11831
   *  beta=0.24107142857142852,pextern=9/28000
   *  beta=0.24107142857142852,pextern=26/66789
   *  beta=0.24107142857142852,pextern=2/4375
   *  beta=0.24107142857142852,pextern=21/40000
   *  beta=0.24107142857142852,pextern=59/99518
   *  beta=0.24107142857142852,pextern=37/56000
   *  beta=0.24107142857142852,pextern=51/70000
   *  beta=0.24107142857142852,pextern=38/47713
   *  beta=0.24107142857142852,pextern=81/93719
   *  beta=0.24107142857142852,pextern=64/68659
   *  beta=0.24107142857142852,pextern=1/1000
   *  beta=0.2607142857142857,pextern=1/20000
   *  beta=0.2607142857142857,pextern=7/59394
   *  beta=0.2607142857142857,pextern=13/70000
   *  beta=0.2607142857142857,pextern=3/11831
   *  beta=0.2607142857142857,pextern=9/28000
   *  beta=0.2607142857142857,pextern=26/66789
   *  beta=0.2607142857142857,pextern=2/4375
   *  beta=0.2607142857142857,pextern=21/40000
   *  beta=0.2607142857142857,pextern=59/99518
   *  beta=0.2607142857142857,pextern=37/56000
   *  beta=0.2607142857142857,pextern=51/70000
   *  beta=0.2607142857142857,pextern=38/47713
   *  beta=0.2607142857142857,pextern=81/93719
   *  beta=0.2607142857142857,pextern=64/68659
   *  beta=0.2607142857142857,pextern=1/1000
   *  beta=0.2803571428571428,pextern=1/20000
   *  beta=0.2803571428571428,pextern=7/59394
   *  beta=0.2803571428571428,pextern=13/70000
   *  beta=0.2803571428571428,pextern=3/11831
   *  beta=0.2803571428571428,pextern=9/28000
   *  beta=0.2803571428571428,pextern=26/66789
   *  beta=0.2803571428571428,pextern=2/4375
   *  beta=0.2803571428571428,pextern=21/40000
   *  beta=0.2803571428571428,pextern=59/99518
   *  beta=0.2803571428571428,pextern=37/56000
   *  beta=0.2803571428571428,pextern=51/70000
   *  beta=0.2803571428571428,pextern=38/47713
   *  beta=0.2803571428571428,pextern=81/93719
   *  beta=0.2803571428571428,pextern=64/68659
   *  beta=0.2803571428571428,pextern=1/1000
   *  beta=0.3,pextern=1/20000
   *  beta=0.3,pextern=7/59394
   *  beta=0.3,pextern=13/70000
   *  beta=0.3,pextern=3/11831
   *  beta=0.3,pextern=9/28000
   *  beta=0.3,pextern=26/66789
   *  beta=0.3,pextern=2/4375
   *  beta=0.3,pextern=21/40000
   *  beta=0.3,pextern=59/99518
   *  beta=0.3,pextern=37/56000
   *  beta=0.3,pextern=51/70000
   *  beta=0.3,pextern=38/47713
   *  beta=0.3,pextern=81/93719
   *  beta=0.3,pextern=64/68659
   *  beta=0.3,pextern=1/1000
## Settings 
type                     |p_periodic               |period                   
-------------------------|-------------------------|-------------------------
28/1                     |1.000                    |28.000                   
28/4                     |0.250                    |7.000                    
No testing (56 days)     |0                        |1                        


## Baseline parameters
parameter           |value               
--------------------|--------------------
G                   |{'numNodes': 1000, 'm': 9, 'scale': 100, 'plot_degree': False}
beta                |0.200               
sigma               |0.192               
gamma               |0.071               
mu_I                |1/1000              
mu_0                |0                   
nu                  |0                   
xi                  |0                   
p                   |0.500               
Q                   |None                
beta_D              |0                   
sigma_D             |0.192               
gamma_D             |0.071               
mu_D                |1/1000              
theta_E             |0                   
theta_I             |0                   
phi_E               |0                   
phi_I               |0                   
psi_E               |0.330               
psi_I               |0.800               
q                   |1/1000              
initI               |0                   
initE               |0                   
initD_E             |0                   
initD_I             |0                   
initR               |0                   
initF               |0                   
p_extern            |1/1000              
p_periodic          |0                   
period              |1                   
batch               |True                
min_time            |0                   
store_Xseries       |False               
count_non_random    |False               
T                   |108                 
verbose             |False               
checkpoints         |None                
stopping            |1st                 
policy              |None                
policyInterval      |1                   
type                |[UNKNOWN]           
runTillEnd          |True                
test_recovered      |True                
initT               |0                   


In [19]:
import pickle
with open(f'{datadir}/to_run.pickle', 'wb') as handle:
    pickle.dump(to_run, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [20]:
import os
os.chdir(datadir)

In [21]:
!pwd

/home/jupyter/covidtesting/data/20200623_12_17_42_400


In [22]:
%%time
!python ../../par_run.py $realizations

Loading to_run
Loaded
Preparing list to run
We have 96 CPUs
Starting execution of 270000 runs
318: beta=0.16249999999999998,pextern=2/4375
318: beta=0.16249999999999998,pextern=2/4375 -- DONE
563: beta=0.3,pextern=2/4375
563: beta=0.3,pextern=2/4375 -- DONE
549: beta=0.2607142857142857,pextern=59/99518
549: beta=0.2607142857142857,pextern=59/99518 -- DONE
276: beta=0.16249999999999998,pextern=38/47713
276: beta=0.16249999999999998,pextern=38/47713 -- DONE
322: beta=0.12321428571428569,pextern=1/1000
322: beta=0.12321428571428569,pextern=1/1000 -- DONE
400: beta=0.06428571428571428,pextern=13/70000
400: beta=0.06428571428571428,pextern=13/70000 -- DONE
919: beta=0.12321428571428569,pextern=21/40000
919: beta=0.12321428571428569,pextern=21/40000 -- DONE
790: beta=0.06428571428571428,pextern=59/99518
790: beta=0.06428571428571428,pextern=59/99518 -- DONE
221: beta=0.06428571428571428,pextern=3/11831
221: beta=0.06428571428571428,pextern=3/11831 -- DONE
890: beta=0.16249999999999998,pexter

In [23]:
os.chdir("/home/jupyter/covidtesting/")


**End of time consuming part**

In [16]:
datadir = 'data/20200623_12_17_42_400' #'data/20200622_21_02_55_400'

In [17]:
datas = []
for i in range(3):
    datas.append(pd.read_pickle(f'{datadir}/data_{i}.zip'))

In [18]:
data = pd.concat(datas)
data


,beta,sigma,gamma,xi,mu_I,mu_0,nu,beta_D,sigma_D,gamma_D,...,meanUndetected,undetected1st,infected1st,totUndetected1st,meanUndetected1st,meanTests,finUndetected,overall_infected,excessRisk,model
0,0.025,0.192308,0.071429,0,0.0004,0,0,0,0.192308,0.071429,...,0.966880,0.0,1.0,27.072650,0.966880,35.714286,0.0,2.0,-70.749129,<models.SEIRSNetworkModel object at 0x7f044046...
1,0.025,0.192308,0.071429,0,0.0004,0,0,0,0.192308,0.071429,...,1.000000,0.0,1.0,7.000000,1.000000,35.714286,0.0,1.0,-22.883137,<models.SEIRSNetworkModel object at 0x7f044046...
2,0.025,0.192308,0.071429,0,0.0004,0,0,0,0.192308,0.071429,...,1.480778,4.0,4.0,82.923549,1.480778,0.000000,4.0,5.0,-15.147699,<models.SEIRSNetworkModel object at 0x7f044046...
3,0.025,0.192308,0.071429,0,0.0004,0,0,0,0.192308,0.071429,...,1.539218,1.0,2.0,43.098110,1.539218,35.714286,1.0,3.0,-74.836714,<models.SEIRSNetworkModel object at 0x7f044046...
4,0.025,0.192308,0.071429,0,0.0004,0,0,0,0.192308,0.071429,...,1.000000,0.0,1.0,7.000000,1.000000,35.714286,0.0,1.0,-100.000000,<models.SEIRSNetworkModel object at 0x7f043f86...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269995,0.300,0.192308,0.071429,0,0.0004,0,0,0,0.192308,0.071429,...,7.781689,13.0,15.0,54.471825,7.781689,35.714286,13.0,15.0,-88.252431,None
269996,0.300,0.192308,0.071429,0,0.0004,0,0,0,0.192308,0.071429,...,136.998173,390.0,390.0,7671.897710,136.998173,0.000000,390.0,740.0,534.789951,None
269997,0.300,0.192308,0.071429,0,0.0004,0,0,0,0.192308,0.071429,...,56.676077,71.0,189.0,1586.930163,56.676077,35.714286,71.0,237.0,100.379760,None
269998,0.300,0.192308,0.071429,0,0.0004,0,0,0,0.192308,0.071429,...,10.905188,25.0,28.0,152.672638,10.905188,35.714286,25.0,33.0,-70.563770,None


In [21]:
data["R"] = data["beta"]*14
data.loc[:,"risk"] = data["meanUndetected1st"]
data["Baseline"] = data["p_extern"]*14*1000


In [23]:
infrequent = data[data.type == "28/1"].groupby(["p_extern","R"]).risk.mean().unstack()
infrequent

R,0.350,0.625,0.900,1.175,1.450,1.725,2.000,2.275,2.550,2.825,3.100,3.375,3.650,3.925,4.200
p_extern,,,,,,,,,,,,,,,
0.000050,1.294162,1.381777,1.416928,1.489091,1.730913,1.789435,1.868282,1.974748,2.096373,2.071419,2.504541,2.646859,2.680955,3.113918,3.070621
0.000118,1.982477,2.044422,2.241074,2.274276,2.537947,2.808099,3.331368,3.331231,3.351176,3.659923,4.066117,4.584737,4.744354,5.450177,6.014407
0.000186,2.634517,2.871593,3.094375,3.296510,3.550131,3.834534,4.122336,4.432912,4.695079,5.106523,6.213326,6.395254,6.882503,7.994188,7.991834
0.000254,3.257284,3.525847,3.747262,4.138242,4.500769,5.247907,5.485949,5.912116,6.680424,7.220233,7.488524,8.337694,8.440955,9.620810,11.033737
0.000321,4.016666,4.431062,4.772821,5.077662,5.506119,6.223617,6.882598,7.566471,7.875378,8.799353,9.350050,10.314513,10.995086,12.516230,13.692528
0.000389,4.899415,5.250938,5.408610,5.939515,6.489923,7.375218,8.197964,8.467773,9.228774,10.128642,11.288384,12.403672,14.603661,14.404656,15.901315
0.000457,5.546815,5.814694,6.367354,7.099455,7.698735,8.542293,9.263337,9.634330,10.997387,12.021651,12.686259,14.017900,15.454073,17.330612,18.411091
0.000525,6.155102,6.952243,7.282033,8.060983,8.660676,9.640157,10.493869,11.373273,12.381330,13.810549,15.028225,16.208075,17.221837,19.749531,20.255989
0.000593,6.887705,7.506716,8.355766,9.084315,9.840027,10.779373,11.775557,13.345365,14.109305,15.220552,16.472228,18.229963,19.603963,21.764198,24.370267


In [24]:
frequent = data[data.type == "28/4"].groupby(["p_extern","R"]).risk.mean().unstack()

In [48]:
notesting = data[data.type == "No testing (56 days)"].groupby(["p_extern","R"]).risk.mean().unstack()

In [ ]:
notesting


In [32]:
baseline=  data[data.type == "28/4"].groupby(["p_extern","R"]).Baseline.mean().unstack()

In [33]:
panel1 = 100*frequent/baseline
panel1

R,0.350,0.625,0.900,1.175,1.450,1.725,2.000,2.275,2.550,2.825,3.100,3.375,3.650,3.925,4.200
p_extern,,,,,,,,,,,,,,,
0.000050,173.929828,172.214238,184.669557,190.290768,202.509581,208.025729,207.128194,215.880883,212.457598,221.388037,233.001058,235.076818,254.601679,258.073757,272.027552
0.000118,101.522539,104.169018,108.743510,113.559520,113.021505,115.295722,117.956432,126.696678,128.616966,128.124664,131.148428,135.102851,134.327625,143.504319,140.864202
0.000186,78.760397,79.789019,85.863682,86.874558,87.636916,91.726918,94.907036,95.379404,96.592134,101.416366,100.938452,109.187022,104.786617,109.605805,111.904663
0.000254,66.783372,68.415028,72.868362,72.059927,74.390063,78.358176,78.072500,81.086157,84.007914,84.773123,88.677006,87.482188,88.327677,89.552439,91.664031
0.000321,60.082613,62.894652,63.164015,62.731641,65.919955,70.076449,69.313185,74.434911,70.946386,76.270852,74.983273,76.604556,74.504625,77.985338,82.730392
0.000389,54.373708,56.956589,58.049250,60.358466,60.837552,61.383184,62.652158,61.836126,66.351120,68.241490,70.458211,69.983484,69.501552,75.054899,75.524975
0.000457,52.531637,52.540141,51.591101,56.171794,56.402365,58.897297,58.881971,59.993221,60.886373,61.063044,62.920047,64.104919,61.842729,67.063441,67.264483
0.000525,50.195335,50.093138,50.760912,49.638999,52.097353,56.682787,54.002036,54.659837,55.872664,55.724070,57.791658,60.796566,60.260458,62.508177,64.480366
0.000593,46.233133,46.953227,48.723363,48.856381,48.921552,50.057360,51.257560,52.666707,53.920245,53.357463,53.532036,57.210785,55.657043,60.518426,61.618845


In [34]:
panel2 = 100*frequent/infrequent

In [49]:
panel3 = 100*frequent/notesting

In [50]:
panel3

R,0.350,0.625,0.900,1.175,1.450,1.725,2.000,2.275,2.550,2.825,3.100,3.375,3.650,3.925,4.200
p_extern,,,,,,,,,,,,,,,
0.000050,91.516707,78.031308,69.002133,63.569024,55.893202,44.922054,39.603015,32.607607,24.752581,23.665594,19.357654,14.700147,13.123818,9.887462,10.154086
0.000118,70.729039,61.429149,52.758035,44.941881,36.328669,31.538349,25.704533,22.779291,17.137486,13.654444,11.053126,9.801720,7.775200,6.544926,5.265870
0.000186,58.532815,49.363180,46.238344,37.547490,29.754299,26.093591,20.446474,18.016793,13.631449,11.483127,9.553961,8.126908,6.399918,5.762933,4.470658
0.000254,51.013177,44.427256,38.826936,32.768206,26.629139,23.047006,18.184762,15.556392,12.165251,10.297191,8.288345,6.861489,5.773544,4.860105,4.162799
0.000321,47.438234,42.153565,34.268265,28.439533,23.216962,19.364823,16.135139,14.322266,11.243855,9.245612,7.407216,6.251438,5.048617,4.463302,4.049480
0.000389,42.748037,38.105385,31.975441,27.507646,22.117100,17.861187,14.979522,11.830305,10.106835,8.632619,7.245526,5.956705,4.993217,4.666770,3.967595
0.000457,41.518930,36.031135,28.365913,25.293007,21.299446,17.665815,13.954569,11.599055,9.287177,7.559559,6.568603,5.684138,4.644905,4.237795,3.759641
0.000525,40.003514,34.055056,28.444571,22.392541,19.815658,17.195881,13.145329,10.667120,9.257693,7.340719,6.462663,5.508649,4.634269,4.218989,3.832332
0.000593,36.717260,31.490561,26.864157,22.864771,18.570904,15.267242,12.514888,10.613308,8.782242,7.218912,6.080700,5.467861,4.573090,4.293193,3.870665


In [ ]:
import seaborn as sns

In [ ]:
L1 = sns.color_palette("RdBu_r", 100).as_hex()
greens = list(reversed(sns.color_palette("BuGn_d",50).as_hex()))
L = greens+L1[:50]
for i in range(50,100):
    L+= [L1[i]]*4
from matplotlib.colors import ListedColormap
cmap = ListedColormap(L)
cmap

In [ ]:
from matplotlib.ticker import StrMethodFormatter 
size = 25
titlesize = 30

fig , axes = plt.subplots(1,3, sharex=True, sharey= True, figsize= (20,10))
cbar_ax = fig.add_axes([1.01, 0.2, .03, .7])
axes[0].set_title("a) 28/4 vs. baseline",loc="left",fontsize=int(titlesize))
#axes[0].title.set_text('28/4 vs baseline')
#cmap = sns.diverging_palette(150, 275, sep=1,s=80, l=55, n=25)



color = "white"
sns.heatmap(panel1,ax=axes[0],cbar=False, vmin=0, annot=False, vmax=300, cmap=cmap)# annot=True,fmt='.0f' ,cmap=cmap) # annot_kws={'color':color}

axes[1].set_title('b) 28/4 vs 28/1', loc='left',fontsize=int(titlesize))
sns.heatmap(panel2,ax=axes[1],cbar=False, vmin=0, vmax=300, cmap=cmap)# annot=True,fmt='.0f', cmap=cmap)
axes[2].set_title('c) 28/4 vs No Testing', loc='left',fontsize=int(titlesize))
sns.heatmap(panel3,ax=axes[2],cbar=True, vmin=0, vmax=300,  cbar_ax = cbar_ax ,cmap=cmap) # annot=True,fmt='.0f', cmap=cmap)

cbar_ax.tick_params(labelsize=20)
def f(x, pos):
    if pos % 2: return ""
    p= list(np.linspace(0.00005,0.001,15))[pos]
    return f"{p:.5f}"


def g(x,pos):
    if pos % 2: return ""
    beta = list(np.linspace(0.025,0.3,15))[pos]
    return f"{beta*14:.2f}"
    
import matplotlib 

f_ = matplotlib.ticker.FuncFormatter(f)
g_ = matplotlib.ticker.FuncFormatter(g)

for i in range(3):
    axes[i].set_xlabel('' if i!=1 else 'Internal repdroductive number / R', fontsize=size)
    axes[i].set_ylabel('' if i else 'Probability of infection', fontsize=size)
    axes[i].tick_params(axis='x', labelsize=size ) 
    axes[i].tick_params(axis='y', labelsize=size ) 
    axes[i].yaxis.set_major_formatter(f_)
    axes[i].xaxis.set_major_formatter(g_)

cbar = axes[2].collections[0].colorbar
cbar_ax.set_yticklabels([f"{int(i)}%" for i in cbar.get_ticks()]) # set ticks of your format

    
fig.tight_layout() #rect=[0, 0, .9, 1])

![](heatmap.png)